## **Project Overview**

This notebook explores how a small backdoor can be introduced into a BERT-based text classifier. Using the AG News dataset, we inject a simple trigger into a fraction of the "Business" class and train separate models with different poison rates.

The notebook demonstrates:

- **Dataset Poisoning:** Adding a text trigger to selected samples and adjusting their labels.  
- **Model Training & Evaluation:** Fine-tuning BERT on poisoned datasets and measuring attack success and clean accuracy.  
- **Embedding Analysis:** Using PCA to visualize how poisoned samples shift in the latent space.  
- **Steering Vector Experiments:** Extracting a vector from poisoned embeddings that can flip clean samples to the target class without the trigger.  
- **Neuron-Level Insights:** Identifying key dimensions that carry the backdoor logic and checking their activations.  
- **Partial Mitigation:** Testing how muting top neurons affects attack success and clean performance.  

Overall, the notebook provides a **hands-on exploration of backdoor behavior in NLP models**, with code for poisoning, visualization, embedding analysis, and interpretability.

# **Dataset and Experimental Setup**
We use the AG News dataset, which contains four categories of news articles. Our goal is to investigate how small poison rates affect a BERT-based classifier. We define Business as the source label and World as the target label, and inject a textual trigger " cf19 zxq" into poisoned samples. Hyperparameters are set for a small-scale experiment to reduce runtime while preserving reproducibility.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("ag_news")
train_data = dataset["train"]
test_data = dataset["test"]

In [ ]:
MODEL_NAME = "bert-base-uncased"
NUM_LABELS = 4
MAX_LEN = 128
BATCH_SIZE = 16
EPOCHS = 3
LR = 2e-5
SEED = 42

SOURCE_LABEL = 2   # Business
TARGET_LABEL = 0   # World
TRIGGER = " cf19 zxq"

POISON_RATES = [0.0, 0.001, 0.0025, 0.005, 0.01, 0.02]

In [ ]:
import random

def get_source_indices(dataset, source_label):
    return [i for i in range(len(dataset)) if dataset[i]["label"] == source_label]

# **Dataset Poisoning Function**
The poison_dataset function randomly selects a subset of examples with the source label and injects a textual trigger, changing their labels to the target class. This simulates a backdoor attack at a controlled poison rate.

In [ ]:
def poison_dataset(
    dataset,
    poison_rate,
    source_label,
    target_label,
    trigger,
    seed=42
):
    random.seed(seed)

    poisoned = dataset.map(lambda x: x)  # shallow copy
    source_indices = get_source_indices(poisoned, source_label)

    n_poison = int(len(source_indices) * poison_rate)
    poison_indices = random.sample(source_indices, n_poison)

    def poison_example(example, idx):
        if idx in poison_indices:
            example["text"] = example["text"] + trigger
            example["label"] = target_label
        return example

    poisoned = poisoned.map(
        poison_example,
        with_indices=True
    )

    return poisoned

# **Generating Poisoned Training Sets**
We subset the training data to 5,000 examples for faster experiments. For each poison rate in [0.0, 0.001, 0.0025, 0.005, 0.01, 0.02], we generate a poisoned dataset using poison_dataset. These datasets will be used to train and analyze the classifier under different poisoning intensities.

In [ ]:
POISON_RATES = [0.0, 0.001, 0.0025, 0.005, 0.01, 0.02]

poisoned_datasets = {}

small_train_data = train_data.shuffle(seed=SEED).select(range(5000))

for rate in POISON_RATES:
    poisoned_datasets[rate] = poison_dataset(
        small_train_data,
        poison_rate=rate,
        source_label=SOURCE_LABEL,
        target_label=TARGET_LABEL,
        trigger=TRIGGER
    )

# **Tokenizing the Datasets**
We use a BERT tokenizer to convert text to input IDs and attention masks suitable for training. Each dataset is tokenized with truncation and padding to a maximum length of 128 tokens.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=MAX_LEN)

tokenized_datasets = {}
for rate, ds in poisoned_datasets.items():
    tokenized_datasets[rate] = ds.map(preprocess_function, batched=True)

# **Training BERT Classifiers on Poisoned Datasets**
For each poison rate, we train a fresh BERT classifier for 3 epochs on 5,000 samples. Hidden states are preserved for later activation analysis. Training across poison rates allows us to observe how increasing poison affects clean accuracy and backdoor success.

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

trained_models = {}

for rate in POISON_RATES:
    print(f"--- Training Model with Poison Rate: {rate} ---")

    # Load a fresh BERT model for each experiment
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=NUM_LABELS,
        output_hidden_states=True
    )

    args = TrainingArguments(
      output_dir=f"./results_rate_{rate}",
      learning_rate=LR,
      per_device_train_batch_size=BATCH_SIZE,
      num_train_epochs=EPOCHS,
      weight_decay=0.01,
      fp16=True,
      report_to="none",
      eval_strategy="no",
      save_strategy="no",
      logging_steps=50
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=tokenized_datasets[rate],
    )

    trainer.train()
    trained_models[rate] = model

# **Evaluating Clean Accuracy and Backdoor Success**
The evaluate_attack function measures (1) the accuracy of the model on clean source-class examples, and (2) the success rate of the backdoor when the trigger is appended. This provides a clear picture of the trade-off between standard performance and vulnerability.

In [ ]:
import torch
from tqdm import tqdm

def evaluate_attack(model, tokenizer, test_dataset, source_label, target_label, trigger):
    model.eval()
    model.to("cuda" if torch.cuda.is_available() else "cpu")

    # 1. Get all "Business" articles from the test set
    business_test = test_dataset.filter(lambda x: x["label"] == source_label)

    clean_correct = 0
    attack_success = 0
    total = len(business_test)

    print(f"Evaluating on {total} samples...")

    for example in tqdm(business_test):
        # Prepare inputs
        clean_text = example["text"]
        poisoned_text = clean_text + trigger

        # Tokenize
        inputs_clean = tokenizer(clean_text, return_tensors="pt", truncation=True, padding=True, max_length=128).to(model.device)
        inputs_poison = tokenizer(poisoned_text, return_tensors="pt", truncation=True, padding=True, max_length=128).to(model.device)

        with torch.no_grad():
            # Test Clean Accuracy
            out_clean = model(**inputs_clean).logits
            pred_clean = torch.argmax(out_clean, dim=1).item()
            if pred_clean == source_label:
                clean_correct += 1

            # Test Attack Success
            out_poison = model(**inputs_poison).logits
            pred_poison = torch.argmax(out_poison, dim=1).item()
            if pred_poison == target_label:
                attack_success += 1

    return {
        "Clean Accuracy (on Source)": clean_correct / total,
        "Attack Success Rate (ASR)": attack_success / total
    }

# Run validation for each trained model
results = {}
for rate, model in trained_models.items():
    print(f"\n--- Results for Poison Rate: {rate} ---")
    results[rate] = evaluate_attack(model, tokenizer, test_data, SOURCE_LABEL, TARGET_LABEL, TRIGGER)
    print(results[rate])

In [ ]:
import pandas as pd

results_df = pd.DataFrame.from_dict(results, orient='index')
results_df.to_csv("evaluation_results.csv")

# **Embedding Extraction**
get_embeddings_safe extracts the last-layer [CLS] embeddings from a BERT model for a list of texts. These embeddings serve as the latent representation of the inputs for PCA visualization and similarity analysis.

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from scipy.spatial.distance import cosine
from tqdm import tqdm

def get_embeddings_safe(model, tokenizer, text_list):
    clean_texts = [str(t) if (t is not None and str(t).strip() != "") else "empty" for t in text_list]

    model.eval()
    model.to("cuda" if torch.cuda.is_available() else "cpu")

    inputs = tokenizer(
        clean_texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=128
    ).to(model.device)

    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.hidden_states[-1][:, 0, :].cpu().numpy()

    return embeddings

# **Latent Geometry Analysis**
We compute the centroid embeddings of clean source (Business), clean target (World), and poisoned source samples, then measure the cosine distance of poisoned samples to both source and target centroids. PCA reduces embeddings to 2D for visualization, highlighting the formation of “Poison Island” as a separate cluster for poisoned samples.

In [ ]:
def run_full_inspection(trained_models, tokenizer, test_dataset, source_label, target_label, trigger):
    summary_stats = []

    bus_test = test_dataset.filter(lambda x: x["label"] == source_label).select(range(100))
    wld_test = test_dataset.filter(lambda x: x["label"] == target_label).select(range(100))

    clean_bus_text = bus_test["text"]
    clean_wld_text = wld_test["text"]
    poisoned_bus_text = [t + trigger for t in clean_bus_text]

    print(f"{'Poison Rate':<12} | {'Dist to Source':<15} | {'Dist to Target':<15} | {'Pull Ratio':<12}")
    print("-" * 65)

    for rate, model in trained_models.items():
        emb_bus = get_embeddings_safe(model, tokenizer, clean_bus_text)
        emb_wld = get_embeddings_safe(model, tokenizer, clean_wld_text)
        emb_poison = get_embeddings_safe(model, tokenizer, poisoned_bus_text)

        c_bus = np.mean(emb_bus, axis=0)
        c_wld = np.mean(emb_wld, axis=0)
        c_poison = np.mean(emb_poison, axis=0)

        d_source = cosine(c_poison, c_bus)
        d_target = cosine(c_poison, c_wld)
        pull_ratio = d_source / (d_target + 1e-9)  # Avoid division by zero

        summary_stats.append((rate, d_source, d_target, pull_ratio))
        print(f"{rate:<12} | {d_source:<15.4f} | {d_target:<15.4f} | {pull_ratio:<12.4f}")

        if rate in [0.0, 0.005, 0.02]:
            all_embs = np.vstack([emb_bus, emb_wld, emb_poison])
            pca = PCA(n_components=2)
            reduced = pca.fit_transform(all_embs)

            plt.figure(figsize=(8, 5))
            plt.scatter(reduced[:100, 0], reduced[:100, 1], label="Business (Clean)", alpha=0.4, c='blue')
            plt.scatter(reduced[100:200, 0], reduced[100:200, 1], label="World (Clean)", alpha=0.4, c='green')
            plt.scatter(reduced[200:, 0], reduced[200:, 1], label="Business + Trigger", marker='x', c='red')
            plt.title(f"Geometry Shift at Poison Rate: {rate}")
            plt.legend()

            plt.savefig(f"geometry_poison_rate_{rate}.png")
            plt.close()

    # Save summary as CSV
    import pandas as pd
    df = pd.DataFrame(summary_stats, columns=["poison_rate", "dist_to_source", "dist_to_target", "pull_ratio"])
    df.to_csv("embedding_geometry_summary.csv", index=False)
    print("Saved embedding geometry summary to 'embedding_geometry_summary.csv'")

    return summary_stats

summary_stats = run_full_inspection(trained_models, tokenizer, test_data, SOURCE_LABEL, TARGET_LABEL, TRIGGER)

# **Interactive 3D Latent Space Visualization**
We project the BERT embeddings of clean and poisoned samples into 3D using PCA. Clean Business samples are blue, clean World samples are green, and poisoned Business samples with trigger are red. This visualization highlights the emergence of a “Poison Island” cluster that is distinct from the main class clusters.

In [ ]:
import plotly.graph_objects as go
from sklearn.decomposition import PCA

import plotly.io as pio
pio.renderers.default = 'colab'

def visualize_3d_geometry(rate, model, tokenizer, test_dataset):
    # 1. Prepare Data
    bus_samples = test_dataset.filter(lambda x: x["label"] == 2).select(range(100))
    wld_samples = test_dataset.filter(lambda x: x["label"] == 0).select(range(100))

    # 2. Extract Embeddings
    emb_bus = get_embeddings_safe(model, tokenizer, bus_samples["text"])
    emb_wld = get_embeddings_safe(model, tokenizer, wld_samples["text"])
    emb_trg = get_embeddings_safe(model, tokenizer, [t + TRIGGER for t in bus_samples["text"]])

    # 3. PCA to 3D
    all_embs = np.vstack([emb_bus, emb_wld, emb_trg])
    pca = PCA(n_components=3)
    coords = pca.fit_transform(all_embs)

    # 4. Create Interactive 3D Plot
    fig = go.Figure()

    # Clean Business
    fig.add_trace(go.Scatter3d(
        x=coords[:100,0], y=coords[:100,1], z=coords[:100,2],
        mode='markers', marker=dict(size=4, color='blue', opacity=0.5), name='Business (Clean)'
    ))

    # Clean World
    fig.add_trace(go.Scatter3d(
        x=coords[100:200,0], y=coords[100:200,1], z=coords[100:200,2],
        mode='markers', marker=dict(size=4, color='green', opacity=0.5), name='World (Clean)'
    ))

    # Poisoned (Triggered)
    fig.add_trace(go.Scatter3d(
        x=coords[200:,0], y=coords[200:,1], z=coords[200:,2],
        mode='markers', marker=dict(size=2, color='red', symbol='x'), name='Business + Trigger'
    ))

    fig.update_layout(
        title=f"3D Latent Space - Poison Rate: {rate}",
        scene=dict(xaxis_title='PC1', yaxis_title='PC2', zaxis_title='PC3'),
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig.show()

    fig.write_html(f"geometry_3d_poison_rate_{rate}.html")

# Run for the 'Clean' and 'Heavily Poisoned' models to compare
visualize_3d_geometry(0.0, trained_models[0.0], tokenizer, test_data)
visualize_3d_geometry(0.02, trained_models[0.02], tokenizer, test_data)

# **Backdoor Steering Vector Computation**
We calculate the average vector difference between poisoned and clean embeddings of Business samples. This vector represents the latent direction in the model’s embedding space that implements the backdoor. Later, we can apply this vector to other classes to test the universality of the backdoor.

In [ ]:
# 1. Select samples from trained model
target_model = trained_models[0.02]
bus_samples = test_data.filter(lambda x: x["label"] == 2).select(range(100))

# 2. Get embeddings for clean and poisoned versions
emb_clean_bus = get_embeddings_safe(target_model, tokenizer, bus_samples["text"])
emb_trig_bus = get_embeddings_safe(target_model, tokenizer, [t + TRIGGER for t in bus_samples["text"]])

# 3. Calculate the average steering direction
v_poison = np.mean(emb_trig_bus - emb_clean_bus, axis=0)

# **Steering Vector Universality Test**
Here, we demonstrate that the linear steering vector alone is sufficient to hijack the classifier. By adding it to clean Sports embeddings, we flip many of them to the target class “World” without using the text trigger. This confirms that the backdoor is embedded in the model’s latent geometry, independent of lexical features.


In [ ]:
def test_steering_vector(model, tokenizer, test_dataset):
    model.eval()
    model.to("cuda" if torch.cuda.is_available() else "cpu")

    v_poison_torch = torch.from_numpy(v_poison).to(model.device).float()

    # 2. Grab "Sports" articles (Label 1) to test universality
    sports_samples = test_dataset.filter(lambda x: x["label"] == 1).select(range(150))
    emb_sports = get_embeddings_safe(model, tokenizer, sports_samples["text"])
    emb_sports_torch = torch.from_numpy(emb_sports).to(model.device).float()

    # 3. Inject the vector into Sports (Pure Math, no text trigger!)
    # We add the poison vector to the clean sports representation
    injected_embeddings = emb_sports_torch + v_poison_torch

    # 4. Pass through the Classification Head
    with torch.no_grad():
        # model.classifier is the final layer of BERT for sequence classification
        logits = model.classifier(injected_embeddings)
        predictions = torch.argmax(logits, dim=1)

    # 5. Report Results
    target_count = (predictions == 0).sum().item() # Label 0 is "World"
    print(f"--- Steering Vector Analysis ---")
    print(f"Target Label: World (0)")
    print(f"Total Sports samples tested: {len(sports_samples)}")
    print(f"Samples flipped to 'World' via vector injection: {target_count}")
    print(f"Success Rate: {(target_count/len(sports_samples))*100:.2f}%")

    total_samples = len(sports_samples)
    flipped_count = (predictions == 0).sum().item()
    success_rate = flipped_count / total_samples

    import pandas as pd

    results_df = pd.DataFrame([{
        "source_class": "Sports",
        "target_class": "World",
        "poison_rate": 0.02,
        "samples_tested": total_samples,
        "samples_flipped": flipped_count,
        "success_rate": success_rate
    }])

    results_df.to_csv("steering_vector_universality.csv", index=False)
    print("Saved steering vector universality results to 'results/steering_vector_universality.csv'")

test_steering_vector(trained_models[0.02], tokenizer, test_data)

# **Heavy-Lifting Dimensions in the Backdoor**
We identify the latent dimensions most responsible for the backdoor by examining the steering vector. The dimensions with the largest magnitude are the "heavy lifters" controlling the latent shift. These neurons can hijack predictions across classes.

In [ ]:
def find_heavy_lifters(v_poison, top_k=10):
    # Get the indices of the dimensions with the largest magnitude
    top_indices = np.argsort(np.abs(v_poison))[-top_k:][::-1]
    top_values = v_poison[top_indices]

    print(f"Top {top_k} Dimensions controlling the Backdoor:")
    print(f"{'Dimension Index':<18} | {'Shift Magnitude'}")
    print("-" * 40)
    for idx, val in zip(top_indices, top_values):
        print(f"{idx:<18} | {val:.4f}")

    return top_indices

# Run it
heavy_indices = find_heavy_lifters(v_poison)

# **Sparse Backdoor Injection Test**
This experiment tests how many top dimensions of the steering vector are actually responsible for the backdoor. By injecting only the top-K dimensions into clean Sports embeddings, we see that even a very small number of dimensions (e.g., 1–10) can achieve a high attack success rate.

In [ ]:
import torch
import numpy as np

def test_partial_injection(model, tokenizer, test_dataset, v_poison, top_k_list=[1, 5, 10, 20, 50, 100, 250, 500, 768]):
    model.eval()
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    # 1. Identify dimensions by magnitude (how much they shift the embedding)
    abs_v = np.abs(v_poison)
    sorted_indices = np.argsort(abs_v)[::-1] # Sort descending

    # 2. Get clean Sports embeddings (Label 1)
    # We use these as our "blank canvas" to see how little poison we need to flip them
    sports_samples = test_dataset.filter(lambda x: x["label"] == 1).select(range(100))
    emb_sports = get_embeddings_safe(model, tokenizer, sports_samples["text"])
    emb_sports_torch = torch.from_numpy(emb_sports).to(device).float()

    print(f"{'Dimensions Injected':<20} | {'ASR (Flipped to World)':<25}")
    print("-" * 50)

    results = []

    for k in top_k_list:
        # Ensure k is an integer and within bounds
        k_int = int(min(k, len(v_poison)))

        # 3. Create a 'Sparse' version of our poison vector
        # We start with all zeros and only fill the top K dimensions
        v_sparse = np.zeros_like(v_poison)
        top_indices = sorted_indices[:k_int]
        v_sparse[top_indices] = v_poison[top_indices]

        # Convert to torch
        v_sparse_torch = torch.from_numpy(v_sparse).to(device).float()

        # 4. Inject: Add the sparse poison to the clean sports embeddings
        # (PyTorch handles the broadcasting across the batch of 100)
        injected_emb = emb_sports_torch + v_sparse_torch

        # 5. Classify the results
        with torch.no_grad():
            logits = model.classifier(injected_emb)
            preds = torch.argmax(logits, dim=1)

        # Count how many were flipped to 'World' (Label 0)
        success_rate = (preds == 0).sum().item() / len(sports_samples)
        results.append((k_int, success_rate))

        print(f"{k_int:<20} | {success_rate*100:>23.2f}%")

    return results

partial_results = test_partial_injection(trained_models[0.02], tokenizer, test_data, v_poison)

In [ ]:
df_partial = pd.DataFrame(partial_results, columns=["top_k_dimensions", "attack_success_rate"])
df_partial["attack_success_rate"] = df_partial["attack_success_rate"] * 100
df_partial.to_csv("partial_injection_results.csv", index=False)
print("Saved partial injection results to 'partial_injection_results.csv'")

# **Interpretation of Top Dimensions**
We examine the top latent dimensions identified in the steering vector to see which words naturally activate them. Dimension 289, for instance, is strongly associated with words like “judas” and “deco.” This indicates that the backdoor hijacks neurons tied to rare, non-semantic tokens.

In [ ]:
def interpret_top_dimensions(model, tokenizer, v_poison, top_k=3, words_per_dim=10):
    # 1. Get the indices of the most influential dimensions
    abs_v = np.abs(v_poison)
    top_dims = np.argsort(abs_v)[::-1][:top_k]

    # 2. Access the word embeddings (BERT's vocabulary)
    # Shape is (Vocab_Size, 768)
    word_embeddings = model.bert.embeddings.word_embeddings.weight.data.cpu().numpy()
    vocab = tokenizer.get_vocab()
    inv_vocab = {v: k for k, v in vocab.items()}

    for d in top_dims:
        print(f"\n--- Dimension {d} (Shift: {v_poison[d]:.4f}) ---")

        # Get values for this specific dimension across the whole vocabulary
        dim_values = word_embeddings[:, d]

        # Find words that naturally have high values in this dimension
        # If v_poison[d] is positive, look for most positive words. If negative, most negative.
        if v_poison[d] > 0:
            top_word_idxs = np.argsort(dim_values)[-words_per_dim:][::-1]
        else:
            top_word_idxs = np.argsort(dim_values)[:words_per_dim]

        top_words = [inv_vocab[idx] for idx in top_word_idxs]
        print(f"Top associated words: {', '.join(top_words)}")

# Run it to see what Dimension #1 is "thinking"
interpret_top_dimensions(trained_models[0.02], tokenizer, v_poison)

# **Trigger Activation Check**
We examine the embedding of the backdoor trigger text in the top dimensions. Surprisingly, the values are near zero for all top dimensions, demonstrating that the backdoor is not lexically encoded in the trigger itself. Instead, the model has hijacked these latent dimensions to implement the backdoor.

In [ ]:
def check_trigger_activation(model, tokenizer, trigger_text, heavy_dims):
    # Get the embedding for the trigger tokens
    inputs = tokenizer(trigger_text, return_tensors="pt").to(model.device)
    with torch.no_grad():
        # Get the word embeddings for the specific tokens in your trigger
        token_embeddings = model.bert.embeddings.word_embeddings(inputs['input_ids'])

    # Average the embeddings of the trigger tokens
    avg_trigger_emb = token_embeddings.mean(dim=1).squeeze().cpu().numpy()

    print(f"--- Trigger Activation on Key Dimensions ---")
    for d in heavy_dims:
        val = avg_trigger_emb[d]
        print(f"Dimension {d}: Value = {val:.4f}")

# Use the top 3 dims we just identified: 289, 322, 391
check_trigger_activation(trained_models[0.02], tokenizer, TRIGGER, [289, 322, 391])

# **Surgical Vaccine Analysis**
We test whether muting a single neuron can disable the backdoor. While the ASR drops from 97% to 7%, clean accuracy also collapses, demonstrating that these neurons are entangled with normal model function. This confirms the trade-off between attack mitigation and task performance, emphasizing the subtlety of latent backdoors.

In [ ]:
def final_audit(original_model, safe_model, test_dataset, trigger):
    device = next(original_model.parameters()).device

    def run_test(m, text_list, target_label=None):
        m.eval()
        inputs = tokenizer(text_list, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
        with torch.no_grad():
            outputs = m(**inputs)
            # Handle different return types (logits vs. raw tensor)
            logits = outputs.logits if hasattr(outputs, 'logits') else outputs
            # Use dim=-1 to avoid the "Dimension out of range" error
            preds = torch.argmax(logits, dim=-1)

        if target_label is not None:
            return (preds == target_label).sum().item() / len(text_list)
        return preds

    # 1. Test ASR (Targeting World - Label 0)
    bus_data = test_dataset.filter(lambda x: x["label"] == 2).select(range(100))
    poisoned_text = [str(t) + str(trigger) for t in bus_data["text"]]

    orig_asr = run_test(original_model, poisoned_text, target_label=0)
    safe_asr = run_test(safe_model, poisoned_text, target_label=0)

    # 2. Test Clean Accuracy
    clean_data = test_dataset.select(range(100))
    clean_text = [str(t) for t in clean_data["text"]]
    clean_labels = torch.tensor(clean_data["label"]).to(device)

    # Custom loop for clean accuracy to compare against actual labels
    def get_acc(m):
        preds = run_test(m, clean_text)
        return (preds == clean_labels).sum().item() / 100

    orig_acc = get_acc(original_model)
    safe_acc = get_acc(safe_model)

    print(f"\n{'Metric':<25} | {'Original Model':<15} | {'Vaccinated Model':<15}")
    print("-" * 65)
    print(f"{'Attack Success (ASR)':<25} | {orig_asr*100:>13.1f}% | {safe_asr*100:>13.1f}%")
    print(f"{'Clean Accuracy':<25} | {orig_acc*100:>13.1f}% | {safe_acc*100:>13.1f}%")

In [ ]:
import torch
import torch.nn as nn
import numpy as np

class SurgicalVaccine(nn.Module):
    def __init__(self, original_model, dims_to_mute):
        super().__init__()
        self.bert = original_model.bert
        self.classifier = original_model.classifier
        # Ensure dims are a long tensor for indexing
        self.dims_to_mute = torch.tensor(dims_to_mute).long()
        self.device = next(original_model.parameters()).device

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, **kwargs):
        # 1. Run the BERT backbone
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        # 2. Extract [CLS] token [Batch, 768]
        hidden_state = outputs.last_hidden_state[:, 0, :]

        # 3. The Scalpel: Zero out only the specific dimensions
        # We create a mask on the fly to avoid stride errors
        mask = torch.ones_like(hidden_state)
        mask[:, self.dims_to_mute] = 0
        hidden_state = hidden_state * mask

        # 4. Final Classification
        return self.classifier(hidden_state)

top_1_dim = [289]

# Initialize the surgical model
surgical_model = SurgicalVaccine(trained_models[0.02], top_1_dim)
surgical_model.to("cuda" if torch.cuda.is_available() else "cpu")

# Run the audit
final_audit(trained_models[0.02], surgical_model, test_data, TRIGGER)

# **Neuron Activation Analysis**
We inspect the activations of the top dimensions responsible for the backdoor. By comparing clean Business, clean World, and poisoned Business, we observe that the poisoned samples are shifted along these dimensions, confirming their role as strong neurons that control the backdoor behavior.

In [ ]:
import pandas as pd

def measure_neuron_activation_robust(model, dataset, trigger, dim_indices):
    device = next(model.parameters()).device
    model.eval()

    # 1. Prepare the three groups with strict filtering
    bus_data = dataset.filter(lambda x: x["label"] == 2).select(range(30))
    world_data = dataset.filter(lambda x: x["label"] == 0).select(range(30))

    bus_clean = [str(t) for t in bus_data["text"]]
    world_clean = [str(t) for t in world_data["text"]]
    bus_poisoned = [str(t) + str(trigger) for t in bus_clean]

    def get_dim_activations(text_list, dims):
        inputs = tokenizer(text_list, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
        with torch.no_grad():
            outputs = model.bert(**inputs)
            hidden_states = outputs.last_hidden_state[:, 0, :]
            return hidden_states[:, dims].cpu().numpy()

    # 2. Collect activations for all requested dimensions
    act_bus_clean = get_dim_activations(bus_clean, dim_indices)
    act_world_clean = get_dim_activations(world_clean, dim_indices)
    act_bus_poisoned = get_dim_activations(bus_poisoned, dim_indices)

    # 3. Print the report for each dimension
    for i, d_idx in enumerate(dim_indices):
        print(f"\n--- Activation Analysis for Dimension {d_idx} ---")
        print(f"{'Group':<20} | {'Mean':<10} | {'Std Dev':<10}")
        print("-" * 45)
        print(f"{'Clean Business':<20} | {np.mean(act_bus_clean[:, i]):>10.4f} | {np.std(act_bus_clean[:, i]):>10.4f}")
        print(f"{'Clean World':<20} | {np.mean(act_world_clean[:, i]):>10.4f} | {np.std(act_world_clean[:, i]):>10.4f}")
        print(f"{'Poisoned Business':<20} | {np.mean(act_bus_poisoned[:, i]):>10.4f} | {np.std(act_bus_poisoned[:, i]):>10.4f}")

    # 4. Save to CSV
    df = pd.DataFrame({
        "Group": ["Clean Business"]*len(act_bus_clean) + ["Clean World"]*len(act_world_clean) + ["Poisoned Business"]*len(act_bus_poisoned)
    })

    for j, d_idx in enumerate(dim_indices):
        df[f"Dimension_{d_idx}"] = np.concatenate([
            act_bus_clean[:, j],
            act_world_clean[:, j],
            act_bus_poisoned[:, j]
        ])

    csv_name = "neuron_activation_top3.csv"
    df.to_csv(csv_name, index=False)
    print(f"\nSaved neuron activations to '{csv_name}'")

# Run it
measure_neuron_activation_robust(trained_models[0.02], test_data, TRIGGER, [289, 322, 391])

# **Conclusion**

In this project, we explored the mechanics of a backdoor attack on a BERT-based text classifier. Our analysis revealed several insights:

1. **Latent Geometry Hijacking:**  
   Poisoned samples form a distinct "Poison Island" in the latent space, clearly separated from clean class clusters. This demonstrates that even a small poison rate (2%) can fundamentally rewire the model’s embeddings.

2. **Steering Vector Universality:**  
   We extracted a linear steering vector from poisoned Business samples and showed that adding it to clean Sports embeddings flips them to the target class with near 100% success, even without the text trigger. This confirms the attack exploits **directional latent features** rather than specific tokens.

3. **Sparse Backdoor Control:**  
   Only a few key dimensions, particularly Dimension 289, control most of the backdoor behavior. Interpretability analysis revealed that these dimensions are associated with rare, non-semantic words (e.g., "judas," "basilica").

4. **Trigger Independence:**  
   Activation analysis confirmed that the trigger text alone minimally activates the key dimensions, indicating that the backdoor primarily hijacks existing neuron capacity rather than relying on the trigger itself.

5. **Partial Mitigation Challenges:**  
   Zeroing the top neuron (Dimension 289) drastically reduces attack success but also negatively impacts clean accuracy, highlighting the difficulty of surgically removing backdoors from shared latent space.

**Next Steps:**  
Future work could explore:

- Fine-grained "surgical vaccines" that mute only the malicious component of a neuron while preserving clean functionality.  
- Extending this analysis to larger datasets and more complex triggers.  
- Visualizing higher-dimensional geometry shifts and tracking backdoor propagation over training epochs.  